# COVID-19 : INDIA ANALYSIS

> Thank you Kaggle and amazing community for your help/work. This provides current analysis of Cvoid 19 position in India & prediction for all indian states during Polynomial Regression   

![COVID-19](https://techcrunch.com/wp-content/uploads/2020/02/coronavirus.jpg)

## Required imports

In [ ]:
#import libraries 

import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import datetime as dt
#from matplotlib import ticker
import folium
import branca
from datetime import datetime, timedelta,date
import json, requests

from datetime import timedelta
from sklearn.preprocessing import PolynomialFeatures 
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from statsmodels.tsa.api import Holt,SimpleExpSmoothing,ExponentialSmoothing
from sklearn.metrics import mean_squared_error,r2_score
import statsmodels.api as sm
from fbprophet import Prophet
!pip install plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

%matplotlib inline
print("Import libraries")

## COVID19 Cases in India
<hr>

In [ ]:
#Import data 
#State Summary  
india_data_json = requests.get('https://api.rootnet.in/covid19-in/unofficial/covid19india.org/statewise').json()
df_india = pd.io.json.json_normalize(india_data_json['data']['statewise'])
df_india = df_india[df_india.state != 'State Unassigned']
df_india = df_india.reset_index(drop=True)

#State Wise Data
india_data_json = requests.get('https://api.rootnet.in/covid19-in/unofficial/covid19india.org/statewise/history').json()
df_india_history = pd.io.json.json_normalize(india_data_json)
df_india_history = pd.io.json.json_normalize(india_data_json['data']['history'], 'statewise',['day'],  errors="ignore")
#df_india_history = df_india_history.set_index("state")
df_india_history["date"]=pd.to_datetime(df_india_history["day"])
#deleted duplciate column for date 
df_india_history = df_india_history.drop('day', 1)
#counted days 
df_india_history["day_count"]=(df_india_history["date"]-df_india_history["date"][0])
df_india_history["day_count"]=df_india_history["day_count"].dt.days

print("Import data")

# COVID-19 : Statewise view of Covid in India 

In [ ]:
#calculate Mortality 
df_india["Mortality Rate"]= np.round(np.nan_to_num(100*df_india["deaths"]/df_india["confirmed"]),2)
df_india.style.background_gradient(cmap='Blues',subset=["confirmed"])\
                        .background_gradient(cmap='Reds',subset=["deaths"])\
                        .background_gradient(cmap='Greens',subset=["recovered"])\
                        .background_gradient(cmap='Purples',subset=["active"])\
                        .background_gradient(cmap='YlOrBr',subset=["Mortality Rate"])

In [ ]:
# Adding Location data (Latitude,Longitude)
locations = {
    "Kerala" : [10.8505,76.2711],
    "Maharashtra" : [19.7515,75.7139],
    "Karnataka": [15.3173,75.7139],
    "Telangana": [18.1124,79.0193],
    "Uttar Pradesh": [26.8467,80.9462],
    "Rajasthan": [27.0238,74.2179],
    "Gujarat":[22.2587,71.1924],
    "Delhi" : [28.7041,77.1025],
    "Punjab":[31.1471,75.3412],
    "Tamil Nadu": [11.1271,78.6569],
    "Haryana": [29.0588,76.0856],
    "Madhya Pradesh":[22.9734,78.6569],
    "Jammu and Kashmir":[33.7782,76.5762],
    "Ladakh": [34.1526,77.5770],
    "Andhra Pradesh":[15.9129,79.7400],
    "West Bengal": [22.9868,87.8550],
    "Bihar": [25.0961,85.3131],
    "Chhattisgarh":[21.2787,81.8661],
    "Chandigarh":[30.7333,76.7794],
    "Uttarakhand":[30.0668,79.0193],
    "Himachal Pradesh":[31.1048,77.1734],
    "Goa": [15.2993,74.1240],
    "Odisha":[20.9517,85.0985],
    "Andaman and Nicobar Islands": [11.7401,92.6586],
    "Puducherry":[11.9416,79.8083],
    "Manipur":[24.6637,93.9063],
    "Mizoram":[23.1645,92.9376],
    "Assam":[26.2006,92.9376],
    "Meghalaya":[25.4670,91.3662],
    "Tripura":[23.9408,91.9882],
    "Arunachal Pradesh":[28.2180,94.7278],
    "Jharkhand" : [23.6102,85.2799],
    "Nagaland": [26.1584,94.5624],
    "Sikkim": [27.5330,88.5122],
    "Dadra and Nagar Haveli and Daman and Diu":[20.1809,73.0169],
    "Lakshadweep":[10.5667,72.6417]
  #  "Daman and Diu":[20.4283,72.8397]    
}
df_india["Lat"] = ""
df_india["Long"] = ""

for index in df_india.state :
       df_india.loc[df_india.state == index,"Lat"] = locations[index][0]
       df_india.loc[df_india.state == index,"Long"] = locations[index][1]
    
print("Load Coordinates")

# COVID-19 : Graphical View of Covid 19 position in India

In [ ]:
india_map = folium.Map(location=[23,80], zoom_start=4,max_zoom=6,min_zoom=4,height=500,width="80%")
for i in range(0,len(df_india)):
 
    #print(i)
    folium.Circle(
        location=[df_india.iloc[i]['Lat'], df_india.iloc[i]['Long']],
        tooltip = "<h5 style='text-align:center;font-weight: bold'>"+df_india.iloc[i]['state']+"</h5>"+
                    "<hr style='margin:10px;'>"+
                    "<ul style='color: #444;list-style-type:circle;align-item:left;padding-left:20px;padding-right:20px'>"+
        "<li>Confirmed: "+str(df_india.iloc[i]['confirmed'])+"</li>"+
        "<li>Active:   "+str(df_india.iloc[i]['active'])+"</li>"+
        "<li>Recovered:   "+str(df_india.iloc[i]['recovered'])+"</li>"+
        "<li>Deaths:   "+str(df_india.iloc[i]['deaths'])+"</li>"+
        "<li>Mortality Rate:   "+str(np.round(df_india.iloc[i]['Mortality Rate'],2))+"</li>"+
        "</ul>"
        ,
        radius=(int(np.log(df_india.iloc[i]['confirmed']+1)))*20000,
        color='#ff6600',
        fill_color='#ff8533',
        fill=True).add_to(india_map)

india_map

# Machine Learning Predictions (Polynomial) - 30 June 2020


In [ ]:
count_row = df_india.shape[0] 
Prediction_covid = pd.DataFrame()
    
for i in range(count_row):
    train_ml=df_india_history[df_india_history['state']== df_india["state"][i]]

    #Poly tranformation 
    poly_features = PolynomialFeatures(degree = 5) 
    train_poly=poly_features.fit_transform(np.array(train_ml["day_count"]).reshape(-1,1))
    actual_confirmed_poly=train_ml["confirmed"]
    actual_death_poly=train_ml["deaths"]


    #Linear Model to predict 
    linreg_confrimed=LinearRegression(normalize=True)
    linreg_confrimed.fit(train_poly,actual_confirmed_poly)
    linreg_death=LinearRegression(normalize=True)
    linreg_death.fit(train_poly,actual_death_poly)
    
    #Prediction for 30th June 
   
    new_date_poly=poly_features.fit_transform(np.array(108).reshape(-1,1))    
    if linreg_confrimed.predict(new_date_poly)[0] > 500:
        Prediction_covid = Prediction_covid.append({'state': df_india["state"][i], 'confirmed': round(linreg_confrimed.predict(new_date_poly)[0]), 'deaths': round(linreg_death.predict(new_date_poly)[0]) }, ignore_index=True)


Prediction_covid["Mortality Rate"]= np.round(np.nan_to_num(100*Prediction_covid["deaths"]/Prediction_covid["confirmed"]),2)

print("ML Model")


# Machine Learning Predictions - Maharashtra

In [ ]:
fig1 = px.scatter(Prediction_covid, 
                    y= Prediction_covid["Mortality Rate"],
                    x= Prediction_covid["confirmed"],
                    color= "state", 
                    hover_name= "state",
                    color_continuous_scale= px.colors.sequential.Plasma,
                    title= 'COVID-19 Prediction: Covid 19 Cases vs Mortality rate (30 June)',
                    size = round(np.power(Prediction_covid["confirmed"]+1,0.3)-0.5),
                    size_max = 30,
                    height =600,
                    hover_data=["confirmed","deaths"],
                    )
fig1.update_coloraxes(colorscale="hot")
fig1.update(layout_coloraxis_showscale=False)
fig1.update_yaxes(title_text="Mortality Rate (%)")
fig1.update_xaxes(title_text="Covid 19 Predicted Cases")
fig1.show()

In [ ]:
#Filtered data for 1 state = MH
covid_india=df_india_history[df_india_history['state']=="Maharashtra"]
covid_india = covid_india.set_index("date")
covid_india.head()

#95% values to train the model  
train_ml=covid_india.iloc[:int(covid_india.shape[0]*0.95)]
test_ml=covid_india.iloc[int(covid_india.shape[0]*0.95):]

#Poly tranformation 
poly_features = PolynomialFeatures(degree = 5) 
train_poly=poly_features.fit_transform(np.array(train_ml["day_count"]).reshape(-1,1))
test_poly=poly_features.fit_transform(np.array(test_ml["day_count"]).reshape(-1,1))
actual_poly=train_ml["confirmed"]

#Linear Model to predict 
linreg=LinearRegression(normalize=True)
linreg.fit(train_poly,actual_poly)
prediction_poly=linreg.predict(test_poly)
rmse_poly=np.sqrt(mean_squared_error(test_ml["confirmed"],prediction_poly))
print("ML for MH")

# Machine Learning Predictions - Maharashtra


In [ ]:
covid_india=df_india_history[df_india_history['state']=="Maharashtra"]

final_date = covid_india["date"].max()
day_count_temp = covid_india["day_count"].max()

for i in range(1,20):
     date = final_date + timedelta(days=i)
     count_temp = i +  day_count_temp    
     covid_india = covid_india.append({ 'date': date, 'day_count' : count_temp }, ignore_index=True)

comp_data=poly_features.fit_transform(np.array(covid_india["day_count"]).reshape(-1,1))
predictions_poly=linreg.predict(comp_data)
fig=go.Figure()
fig.add_trace(go.Scatter(x=covid_india["date"], y=covid_india["confirmed"],
                    mode='lines+markers',name="Confirmed Cases"))
fig.add_trace(go.Scatter(x=covid_india["date"], y=predictions_poly,
                    mode='lines',name="Predicted Cases",
                    line=dict(color='red', dash='dot')))
fig.update_layout(title="Maharashtra Polynomial Regression Prediction",
                 xaxis_title="Date",yaxis_title="Covid 19 Cases",
                 legend=dict(x=0,y=1,traceorder="normal"))
fig.show()